In [8]:
import numpy as np
import chess
import chess.uci
import json
import time
import random
import sys
import IPython.core.debugger as db
import pickle

In [9]:
def getNormX(position):
    return (position%8.0)/7.0
def getNormY(position):
    return float(position/8)/7.0

In [10]:
def calcGlobalFeatures(board):
    #side2Move
    #THIS IS PASSED DIRECTLY TO SECOND LAYER
    side2Move = [float(board.turn)]
    
    #king positions (this is covered elsewhere, but in a different group)
    #THIS IS PASSED DIRECTLY TO SECOND LAYER
    kingPosition = [getNormX(board.king(chess.WHITE)), 
        getNormY(board.king(chess.WHITE)), 
        getNormX(board.king(chess.BLACK)),
        getNormY(board.king(chess.BLACK))]
    
    #castlingRights
    #THIS IS PASSED DIRECTLY TO SECOND LAYER (I think)
    castlingRights = [float(board.has_kingside_castling_rights(chess.WHITE)), 
        float(board.has_queenside_castling_rights(chess.WHITE)), 
        float(board.has_kingside_castling_rights(chess.BLACK)), 
        float(board.has_queenside_castling_rights(chess.BLACK))]
    
    #numOfEachPieceType 
    #THIS IS PASSED DIRECTLY TO SECOND LAYER
    numOfEachPieceType = [len(board.pieces(chess.PAWN, chess.WHITE))/8.0, 
        len(board.pieces(chess.KNIGHT, chess.WHITE))/2.0,
        len(board.pieces(chess.BISHOP, chess.WHITE))/2.0,
        len(board.pieces(chess.ROOK, chess.WHITE))/2.0,
        len(board.pieces(chess.QUEEN, chess.WHITE))/1.0,
        len(board.pieces(chess.PAWN, chess.BLACK))/8.0, 
        len(board.pieces(chess.KNIGHT, chess.BLACK))/2.0,
        len(board.pieces(chess.BISHOP, chess.BLACK))/2.0,
        len(board.pieces(chess.ROOK, chess.BLACK))/2.0,
        len(board.pieces(chess.QUEEN, chess.BLACK))/1.0]
    return np.array(side2Move+kingPosition+castlingRights+numOfEachPieceType, dtype='float32')

In [11]:
#see variable SEE::SEE_MAT for values (WK is white king i think)
#NormalizeCount(SEE::SEE_MAT[WK] + SEE::SEE_MAT[WK] / 2 - SEE::SEE_MAT[whitePt], SEE::SEE_MAT[WK] * 2)
'''
static const Score SEE_MAT[14] = {
    1500, // WK
    975, // WQ
    500, // WR
    325, // WN
    325, // WB
    100, // WP
}
'''

def getLowestValueAttackerScore(board, position, color):
    attackers = board.attackers(color, position)
    #determine the lowest value attacker
    if not bool(attackers): #no attackers
        value = 2250.0 #makes the score 0
    elif bool(attackers & board.pieces(chess.PAWN, color)):
        value = 100.0
    elif bool(attackers & board.pieces(chess.BISHOP, color)):
        value = 325.0
    elif bool(attackers & board.pieces(chess.KNIGHT, color)):
        value = 325.0
    elif bool(attackers & board.pieces(chess.ROOK, color)):
        value = 500.0
    elif bool(attackers & board.pieces(chess.QUEEN, color)):
        value = 975.0
    else: #Must be a king attacker
        value = 1500.0
    return (2250.0 - value)/3000.0

In [12]:
def calcSquareFeatures(board):
    feats = []
    for position in range(64):
        feats.append(getLowestValueAttackerScore(board, position, chess.WHITE))
        feats.append(getLowestValueAttackerScore(board, position, chess.BLACK))
    return np.array(feats, dtype='float32')

In [13]:
def getSinglePieceFeatures(board, position, exists=True):
    #return array of pieceExists, XYposition, and attackers and defenders
    if exists:
        whiteAttackerScore = getLowestValueAttackerScore(board, 
                                 position, chess.WHITE)
        blackAttackerScore = getLowestValueAttackerScore(board, 
                                 position, chess.BLACK)
        feats = [1.0, getNormX(position), getNormY(position), 
                    whiteAttackerScore, blackAttackerScore]
    else:
        feats = [0.0, 0.0, 0.0, 0.0, 0.0]
    return feats
        
def getPawnFeatures(board, color):
    pawns = board.pieces(chess.PAWN, color)
    #Assign each pawn to a slot based on X position
    slots = []
    unassignedSlots = []
    unassignedPieces = []
    for x in range(8):
        column = pawns & chess.BB_FILES[x]
        if len(column) == 1: #exactly one pawn in column
            position = column.pop()
            slots.append(getSinglePieceFeatures(board, position))
        elif len(column) == 0: #no pawns in column
            slots.append(None)
            unassignedSlots.append(x)
        else: #more than one pawn in column
            position = column.pop()
            slots.append(getSinglePieceFeatures(board, position))
            unassignedPieces.extend(list(column))
    #match unpaired slots to pawns 
    for x in unassignedSlots:
        #make sure there are still extra pawns
        if len(unassignedPieces) > 0:
            #find closest pawn position
            position = min(unassignedPieces, key=lambda pos : abs(x-pos%8))
            slots[x] = getSinglePieceFeatures(board, position)
            unassignedPieces.remove(position)
        else:
            slots[x] = getSinglePieceFeatures(None, None, exists=False)
       
    #any extra pawns after this don't have a slot and are ignored
    #combine slots into a single array and return it

    return [feature for slot in slots for feature in slot]

def getPairPieceFeatures(board, pieceType, color):
    #for rooks, bishops, and knights, there are only 2 slots
    pieces = board.pieces(pieceType, color)
    if len(pieces) == 0:
        feats = getSinglePieceFeatures(None, None, exists=False)*2
    elif len(pieces) == 1:
        position = pieces.pop()
        emptyFeatures = getSinglePieceFeatures(None, None, exists=False)
        existingFeatures = getSinglePieceFeatures(board, position)
        if position%8 <4:
            feats = existingFeatures + emptyFeatures
        else:
            feats = emptyFeatures + existingFeatures
    else:
        position1 = pieces.pop()
        position2 = pieces.pop()
        feats1 = getSinglePieceFeatures(board, position1)
        feats2 = getSinglePieceFeatures(board, position2)
        if position1%8 < position2%8:
            feats = feats1 + feats2
        else:
            feats = feats2 + feats1
    return feats
    
def getQueenFeatures(board, color):
    queens = board.pieces(chess.QUEEN, color)
    if len(queens) > 0:
        position = queens.pop()
        feats = getSinglePieceFeatures(board, position)
    else:
        feats = getSinglePieceFeatures(None, None, exists=False)
    return feats
    
def getKingFeatures(board, color): #maybe not needed
    kings = board.pieces(chess.KING, color)
    if len(kings) > 0:
        position = kings.pop()
        feats = getSinglePieceFeatures(board, position)
    else: #this should never happen
        feats = getSinglePieceFeatures(None, None, exists=False)
    return feats

#the main function for this cell:
def calcPieceFeatures(board):
    pairPieces = [chess.ROOK, chess.BISHOP, chess.KNIGHT]
    pairPiecesFeatures = []
    for pieceType in pairPieces:
        pairPiecesFeatures.extend(getPairPieceFeatures(board, pieceType, chess.WHITE))
        pairPiecesFeatures.extend(getPairPieceFeatures(board, pieceType, chess.BLACK))
    pieceFeatures = np.array(getPawnFeatures(board, chess.WHITE)
                     +getPawnFeatures(board, chess.BLACK)
                     +pairPiecesFeatures
                     +getQueenFeatures(board, chess.WHITE)
                     +getQueenFeatures(board, chess.BLACK)
                     +getKingFeatures(board, chess.WHITE)
                     +getKingFeatures(board, chess.BLACK), dtype='float32')
    
    return pieceFeatures

In [14]:
def checkRestartEngine(force=False):
    if('engine' not in globals()):
        global engine 
        engine = chess.uci.popen_engine("stockfish")
        engine.uci()
        #engine.debug(True)
        global infoHandler
        infoHandler = chess.uci.InfoHandler()
        engine.info_handlers.append(infoHandler)
        global computeTime
        computeTime = 50
    if force or not engine.is_alive():
        engine = chess.uci.popen_engine("stockfish")
        engine.uci()
        #engine.debug(True)
        infoHandler = chess.uci.InfoHandler()
        engine.info_handlers.append(infoHandler)

In [15]:
def getBestMove(board):
    checkRestartEngine()
    engine.ucinewgame()
    engine.position(board)
    possibleMoves = list(board.pseudo_legal_moves)
    possibleMoves.append(chess.Move(0, 0))
    try:
        bestMove = engine.go(searchmoves=possibleMoves, movetime=computeTime)[0]
        engine.stop()
        if bestMove == None: bestMove = chess.Move(0,0)
    except chess.uci.EngineTerminatedException as e:
        #print('Error processing board for best move:')
        #print(board.fen())
        #print(board)
        #print(infoHandler.info)
        #print(e)
        #print('')
        checkRestartEngine(force=True)
        bestMove = None #random.choice(possibleMoves)
    return bestMove

In [16]:
def calcLabel(board, move, bestMove):
    return np.array([bestMove == move], dtype='float32')

In [35]:
def calcMoveRankings(board):
    moves = list(board.pseudo_legal_moves)
    moves.append(chess.Move(0, 0))
    scores = {}
    print 'Num Moves: ', len(moves)
    for move in moves:
        checkRestartEngine()
        engine.ucinewgame()
        engine.position(board)
        try:
            engine.go(searchmoves=[move], movetime=computeTime)[0]
            engine.stop()
            scores[move] = infoHandler.info['score'][1].cp
        except chess.uci.EngineTerminatedException as e:
            #print('Error processing board and move ranking:')
            #print(board.fen())
            #print(board)
            #print(move)
            #print(infoHandler.info)
            #print(e)
            #print('')
            checkRestartEngine(force=True)
            scores[move] = 0
    moves.sort(key=lambda move: scores[move], reverse=True)
    rankings = {}
    for i, move in enumerate(moves):
            rankings[move] = i
    return rankings

In [18]:
def calcMoveFeatures(board, move):
    #from square
    fromPos = move.from_square
    fromX = getNormX(fromPos)
    fromY = getNormY(fromPos)
    #to square
    toPos = move.to_square
    toX = getNormX(toPos)
    toY = getNormY(toPos)
    #piece type
    pieceTypeLabel = board.piece_type_at(fromPos)
    pieceType = [0.0]*6
    if pieceTypeLabel != None:
        pieceType[pieceTypeLabel-1] = 1.0 
    #promotion type (if any)
    promotion = [0.0]*6
    if move.promotion != None:
        promotion[move.promotion-1] = 1.0
    return np.array([fromX, fromY, toX, toY]+pieceType
                    +promotion, 
                    dtype='float32')

In [44]:
def fen2Features(fen):
    checkRestartEngine()
    board = chess.Board(fen)
    nullMove = chess.Move(0, 0) #null move
    #print 'Starting timer'
    #t0 = time.time()
    bestMove = getBestMove(board)
    #t1 = time.time()
    #print 'Best Move', t1-t0
    if bestMove == None: #don't deal with bad boards
        return None
    #moveRankings = calcMoveRankings(board)
    #t0 = time.time()
    #print 'Rankings', t0-t1
    #consider all possible moves (including no move)
    globalFeatures = calcGlobalFeatures(board)
    pieceFeatures = calcPieceFeatures(board)
    squareFeatures = calcSquareFeatures(board)
    #t1 = time.time()
    #print 'Basic Features', t1-t0, '\n'
    features = []
    '''moveFeatures = calcMoveFeatures(board, nullMove)
    moveRanking = np.array([moveRankings[nullMove]/20.0], dtype='float32')
    label = calcLabel(board, nullMove, bestMove)
    
    features = [{'globalFeatures': globalFeatures, 
                 'pieceFeatures':pieceFeatures, 
                 'squareFeatures':np.concatenate((squareFeatures,moveFeatures)),
                 'moveRankings':moveRanking,
                 'labels':label, 
                 'fen':fen, 
                 'move': nullMove}]'''
    possibleMoves = list(board.pseudo_legal_moves)
    possibleMoves.append(nullMove)
    if bestMove in possibleMoves: 
        possibleMoves.remove(bestMove)
    possibleMoves = [random.choice(possibleMoves), bestMove]
        
    for move in possibleMoves:
        globalFeatures = calcGlobalFeatures(board)
        pieceFeatures = calcPieceFeatures(board)
        squareFeatures = calcSquareFeatures(board)
        #t0 = time.time()
        moveFeatures = calcMoveFeatures(board, move)
        #print 'MoveFeatures', time.time()-t0
        moveRanking = np.array([0.0], dtype='float32') #np.array([moveRankings[move]/20.0], dtype='float32')
        label = calcLabel(board, move, bestMove)
        features.append({'globalFeatures': globalFeatures, 
                         'pieceFeatures':pieceFeatures, 
                         'squareFeatures':np.concatenate((squareFeatures,moveFeatures)),
                         'moveRankings':moveRanking,
                         'labels':label,
                         'fen': fen, 
                         'move': move})
    return features


In [39]:
def randomSearchSample(baseFens, depth=1):
    if depth < 1:
        return []
    if type(baseFens) != list:
        baseFens = [baseFens]
    sampledFenSet = []
    fenSet = []
    threshold = 0.95
    for baseFen in baseFens:
        board = chess.Board(baseFen)
        #consider all possible moves (including no move)
        fenSet.append(baseFen)
        if random.random() > threshold:
            sampledFenSet.append(baseFen)
        for move in board.pseudo_legal_moves:
            board.push(move)
            if board.king(chess.WHITE) == None or board.king(chess.BLACK) == None:
                #db.set_trace()
                board.pop()
                continue
            fenSet.append(board.fen())
            if random.random() > threshold:
                sampledFenSet.append(board.fen())
            board.pop()
    sampledFenSet.extend(randomSearchSample(fenSet, depth-1))
    return sampledFenSet
    

In [40]:
#this takes the raw data and preprocesses it into features
#open the raw data
path = '/home/rvansoelen/rbmcData/isc_games/'
files = ['games1.json', 'games2.json', 'games3.json', 'games4.json']
#files = ['games3.json']
start = time.time()
fenSet = []
maxFenSize = 38000000000000
print('Starting...')
for file in files:
    filepath = path+file
    with open(filepath, 'r') as fid:
        numLines = num_lines = sum(1 for line in fid)
    with open(filepath, 'r') as fid:
        dataset = []
        for i, line in enumerate(fid):
            if i%100 ==0: 
                print'Game', i, ' out of ', numLines
                print(time.time()-start)
            game = json.loads(line)
            if game['status'] != 'complete':
                print('Skipping game ', game['id'], ', status is ', game['status'])
                continue
            for j, turn in enumerate(game['gameHistory']):
                if turn['request'].get('action') != None and turn['request']['action']['action'] == 'move':
                    continue
                elif turn['response']['gameInfo'].get('status') == 'over':
                    continue
                fen = turn['truth']['fen']+' '+str(turn['moveNumber'])
                fenSet.extend(randomSearchSample(fen))
                
                #print boardFeatures
                #if len(fenSet)>maxFenSize:
                #    break
                
            #if len(fenSet)>maxFenSize:
            #    break
    #if len(fenSet)>maxFenSize:
    #    break
random.shuffle(fenSet)
print('Done')
print(time.time()-start)

Starting...
Game 0  out of  2730
0.0811991691589
Game 100  out of  2730
10.1423611641
Game 200  out of  2730
19.899490118
Game 300  out of  2730
32.6879580021
Game 400  out of  2730
49.4377830029
Game 500  out of  2730
66.0660030842
Game 600  out of  2730
84.5263521671
Game 700  out of  2730
97.5846610069
Game 800  out of  2730
115.27948308
Game 900  out of  2730
130.013580084
Game 1000  out of  2730
148.906039
Game 1100  out of  2730
168.543838024
Game 1200  out of  2730
188.024811029
Game 1300  out of  2730
203.987953186
Game 1400  out of  2730
219.13399601
Game 1500  out of  2730
235.205385208
Game 1600  out of  2730
249.906585217
Game 1700  out of  2730
267.509331226
Game 1800  out of  2730
285.910587072
Game 1900  out of  2730
304.669063091
Game 2000  out of  2730
320.772544146
Game 2100  out of  2730
331.494326115
Game 2200  out of  2730
337.865991116
Game 2300  out of  2730
339.06976819
Game 2400  out of  2730
347.364105225
Game 2500  out of  2730
354.750653028
Game 2600  out of

In [41]:
print len(fenSet)

640643


In [45]:
print('Starting...')
start = time.time()
checkRestartEngine(force=True)
boardFeatures = []
for i, fen in enumerate(fenSet):
    if i%1000 ==0: 
        print'Board', i, ' out of ', len(fenSet)
        print len(boardFeatures), 'Boards added'
    #print i, ":   ", fen
    features = fen2Features(fen)
    if features != None: #Could be none if stockfish crashed
        boardFeatures.extend(features)
    #else:
    #    print 'Board Skipped: ', fen
print len(boardFeatures), ' board-move samples generated'
outputFile = 'features_AllBoards_EvenClassLabels_V2.pckl'
with open(outputFile, 'wb') as fid:
    pickle.dump(boardFeatures, fid)
print('Done')
print(time.time()-start)

Starting...
Board 0  out of  640643
0 Boards added
Board 1000  out of  640643
1866 Boards added


No handlers could be found for logger "chess.engine"


Board 2000  out of  640643
3752 Boards added
Board 3000  out of  640643
5640 Boards added
Board 4000  out of  640643
7526 Boards added
Board 5000  out of  640643
9424 Boards added
Board 6000  out of  640643
11302 Boards added
Board 7000  out of  640643
13158 Boards added
Board 8000  out of  640643
15036 Boards added
Board 9000  out of  640643
16926 Boards added
Board 10000  out of  640643
18804 Boards added
Board 11000  out of  640643
20674 Boards added
Board 12000  out of  640643
22558 Boards added
Board 13000  out of  640643
24428 Boards added
Board 14000  out of  640643
26306 Boards added
Board 15000  out of  640643
28186 Boards added
Board 16000  out of  640643
30082 Boards added
Board 17000  out of  640643
31972 Boards added
Board 18000  out of  640643
33850 Boards added
Board 19000  out of  640643
35734 Boards added
Board 20000  out of  640643
37600 Boards added
Board 21000  out of  640643
39482 Boards added
Board 22000  out of  640643
41354 Boards added
Board 23000  out of  6406

Board 173000  out of  640643
324456 Boards added
Board 174000  out of  640643
326372 Boards added
Board 175000  out of  640643
328228 Boards added
Board 176000  out of  640643
330108 Boards added
Board 177000  out of  640643
331966 Boards added
Board 178000  out of  640643
333818 Boards added
Board 179000  out of  640643
335712 Boards added
Board 180000  out of  640643
337582 Boards added
Board 181000  out of  640643
339448 Boards added
Board 182000  out of  640643
341322 Boards added
Board 183000  out of  640643
343184 Boards added
Board 184000  out of  640643
345076 Boards added
Board 185000  out of  640643
346966 Boards added
Board 186000  out of  640643
348852 Boards added
Board 187000  out of  640643
350714 Boards added
Board 188000  out of  640643
352588 Boards added
Board 189000  out of  640643
354440 Boards added
Board 190000  out of  640643
356324 Boards added
Board 191000  out of  640643
358168 Boards added
Board 192000  out of  640643
360052 Boards added
Board 193000  out of

Board 341000  out of  640643
639806 Boards added
Board 342000  out of  640643
641700 Boards added
Board 343000  out of  640643
643596 Boards added
Board 344000  out of  640643
645470 Boards added
Board 345000  out of  640643
647372 Boards added
Board 346000  out of  640643
649252 Boards added
Board 347000  out of  640643
651122 Boards added
Board 348000  out of  640643
653022 Boards added
Board 349000  out of  640643
654884 Boards added
Board 350000  out of  640643
656766 Boards added
Board 351000  out of  640643
658652 Boards added
Board 352000  out of  640643
660488 Boards added
Board 353000  out of  640643
662384 Boards added
Board 354000  out of  640643
664264 Boards added
Board 355000  out of  640643
666134 Boards added
Board 356000  out of  640643
667998 Boards added
Board 357000  out of  640643
669858 Boards added
Board 358000  out of  640643
671742 Boards added
Board 359000  out of  640643
673612 Boards added
Board 360000  out of  640643
675490 Boards added
Board 361000  out of

Board 509000  out of  640643
954898 Boards added
Board 510000  out of  640643
956784 Boards added
Board 511000  out of  640643
958648 Boards added
Board 512000  out of  640643
960530 Boards added
Board 513000  out of  640643
962402 Boards added
Board 514000  out of  640643
964280 Boards added
Board 515000  out of  640643
966172 Boards added
Board 516000  out of  640643
968074 Boards added
Board 517000  out of  640643
969956 Boards added
Board 518000  out of  640643
971828 Boards added
Board 519000  out of  640643
973704 Boards added
Board 520000  out of  640643
975572 Boards added
Board 521000  out of  640643
977462 Boards added
Board 522000  out of  640643
979342 Boards added
Board 523000  out of  640643
981200 Boards added
Board 524000  out of  640643
983076 Boards added
Board 525000  out of  640643
984946 Boards added
Board 526000  out of  640643
986846 Boards added
Board 527000  out of  640643
988732 Boards added
Board 528000  out of  640643
990588 Boards added
Board 529000  out of

In [ ]:
print len(boardFeatures)
print sys.getsizeof(boardFeatures)
print float(sys.getsizeof(boardFeatures))/len(boardFeatures)
print boardFeatures[0]

In [ ]:
#chess.Board(game['gameHistory'][12]['truth']['fen']+" 1")
#turn['response']
#game['gameHistory'][1]['response']['gameInfo']
#fen = turn['truth']['fen'] + " 1"
#chess.Board(fen)
#fen2Features(fen)

with open(filepath, 'r') as fid:
    lines = [line for line in fid]
len(lines)*len(fenSet)*3.0/1e6

In [ ]:
#get the boards and moves from each board

#augement data: (does this bias the representation of states??)
	#perform a time limited search on board states from raw data
	#randomly sample from this search to get dataset
	#label board state with best move, either from stockfish, raw data, or other person's bot
#fen
#db.set_trace()

In [ ]:
#partition data into test, validation, and train

#num_lines = sum(1 for line in open(filepath))
#print num_lines
#randomly sort and save samples